Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB10

Use reinforcement learning to devise a tic-tac-toe player.

### Deadlines:

* Submission: [Dies Natalis Solis Invicti](https://en.wikipedia.org/wiki/Sol_Invictus)
* Reviews: [Befana](https://en.wikipedia.org/wiki/Befana)

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [1]:
import numpy as np
import hashlib as hl
import random as rd
from tqdm.auto import tqdm

In [2]:
def check_win(state):
    state=np.array(state).reshape(3,3)
     # Controllare le righe e le colonne
    for i in range(3):
        # Controllare le righe
        if all(state[i, j] == state[i, 0] and state[i, 0] != ' ' for j in range(3)):
            return True
        # Controllare le colonne
        if all(state[j, i] == state[0, i] and state[0, i] != ' ' for j in range(3)):
            return True
    # Controllare le diagonali
    if state[0, 0] == state[1, 1] == state[2, 2] and state[0, 0] != ' ':
        return True
    if state[0, 2] == state[1, 1] == state[2, 0] and state[0, 2] != ' ':
        return True
    return False

            

# Random Agent

In [3]:
class RandomPlayer:
    def __init__(self, symbol):
        self.symbol = symbol

    def make_move(self, state):
        empty_cells = np.where(np.array(state) == ' ')[0]
        action = np.random.choice(len(empty_cells))   
        state[action] = self.symbol
        return state

# QLearningAgent 


In [19]:
from collections import Counter

def check_pattern(state,symbol):
    if symbol=='X':
        opp='O'
    else:
        opp='X'
    for i in [1,2,3,4,5,6,7,8]:
        for p in [0,2,6,8]:
            if i!=p and state[i]==opp and state[p]==symbol:
                return 3 #wherever the opp starts to play take a corner 
    if (state==[' ',' ',' ',' ',symbol,' ',' ',' ',' ']
        or state==[' ',symbol,' ',' ',' ',' ',' ',' ',' ']
        or state==[' ',' ',' ',symbol,' ',' ',' ',' ',' ']
        or state==[' ',' ',' ',' ',' ',symbol,' ',' ',' ']
        or state==[' ',' ',' ',' ',' ',' ',' ',symbol,' ']):
        return -5 #penalize starting from the sides (1,3,5,7) and from the center
    if (state==[symbol,' ',' ',' ',' ',' ',' ',' ',' '] 
       or state==[' ',' ',symbol,' ',' ',' ',' ',' ',' '] 
       or state==[' ',' ',' ',' ',' ',' ',symbol,' ',' ']
       or state==[' ',' ',' ',' ',' ',' ',' ',' ',symbol]) :
        return 3 #encourage starting from the corner
    for o in [1,3,5,7]:
        for p in [0,2,6,8]:
            if (state[o]==opp and state[p]==symbol and (state[2]==symbol 
            or state[6]==symbol or state[8]==symbol) and state.count(' ')==6):
                return 3 #if opp took a side and you already own a corner take one of the other corners
    if (state==[symbol,' ',' ',' ',opp,' ',' ',' ',symbol] 
       or state==[' ',' ',symbol,' ',opp,' ',symbol,' ',' '] 
       or state==[symbol,' ',' ',' ',opp,' ',symbol,' ',' ']
       or state==[' ',' ',symbol,' ',opp,' ',' ',' ',symbol]
       or state==[symbol,' ',symbol,' ',opp,' ',' ',' ',' ']
       or state==[' ',' ',' ',' ',opp,' ',symbol,' ',symbol]) :
        return 3 # if opp took the center take the opposite corner or the same corner in the row/col
    
    for i in range(3):
        c=Counter(state[i:9:3])
        if c["symbol"]==1 and c["opp"]==2:
            return 4 #block opp on col
        c=Counter(state[i:i+3])
        if c["symbol"]==1 and c["opp"]==2:
            return 4 #block opp on rows
    c=Counter(state[0:9:4])
    if c["symbol"]==1 and c["opp"]==2:
        return 4 #block opp on main diag
    c=Counter(state[2:7:2])
    if c["symbol"]==1 and c["opp"]==2:
        return 4 #block opp on secondary diag

    if (state.count(' ')==5 and ((state[0]==state[2]==state[6]==symbol) or (state[0]==state[2]==state[8]==symbol) 
    or (state[0]==state[6]==state[8]==symbol) or (state[2]==state[6]==state[8]==symbol))):
        return 3 #make triangles (take three corners)
    
    return 0.1
    

In [20]:
def reward(state,symbol):
    if state.count(' ') == 0:
        return 3 #draw
    elif check_win(state)==True :
        return 15 #q-learning agent won
    else:
        # check if random player won
        shadow_state = state.copy()
        shadow_state = RandomPlayer('O').make_move(shadow_state)
        if check_win(shadow_state)==True:
            return -15 #q-learning agent lost
    return check_pattern(state,symbol)            

def get_qvalue_max(qtable,state,action):
    best_act = None
    val_max = float('-inf')
    
    if action is not None and (tuple(state),action) not in qtable:
        return action,0
    else:
        for key, val in qtable.items():
            if key[0] == tuple(state) and val > val_max:
                val_max = val
                best_act = key[1]

    return best_act,val_max

def print_board(state):
    state_as_list = [list(state[i:i+3]) for i in range(0, len(state), 3)]
    for row in state_as_list:
        print(row)
    print("-----------------")
    return


In [6]:

QAGENTMODE="RANDOM"

class QLearningAgent():
    def __init__(self, symbol):
        self.symbol = symbol
        self.eps=0.4
    
    def make_move(self, state):
        possible_moves=np.where(np.array(state)==' ')[0]
 
        if QAGENTMODE=="EPSILON_GREEDY":
            # epsilon greedy
            if rd.random() < self.eps:
                action=np.random.choice(possible_moves)
            else:
                if all((tuple(state),val) not in qtable for val in possible_moves)==True:
                    action=np.random.choice(possible_moves)
                else:
                    action,_=get_qvalue_max(qtable,state,None)

        elif QAGENTMODE=="RANDOM":
            action=np.random.choice(possible_moves)
        else:
            print("ERROR MODE NOT FOUND")
            return None
            
        hashable_state=tuple(state)

        if (hashable_state,action) not in qtable:
            #if new state,action tuple is discovered assign as q-value 
            # a random number between -1 and 1
            qtable[(hashable_state,action)]=np.random.uniform(-1,1)
            state[action]=self.symbol #update the state
            return state 
        else:
            #if a value already exists for the state, update the q-value
            state[action]=self.symbol #update the state 
            _,qvalue_max=get_qvalue_max(qtable,tuple(state),action)
            curr_value=qtable[(hashable_state,action)]
            qtable[(hashable_state,action)]=(1-lr)*curr_value+lr*(reward(state,self.symbol)+discount*qvalue_max)
        return state

    def use_only_qtable(self,state):
        action,_=get_qvalue_max(qtable,state,None)
        if action is None: #new tuple state,action not found in training
            action=np.random.choice(np.where(np.array(state)==' ')[0])
        state[action]=self.symbol
        return state




# Training

In [26]:
lr=0.01
discount=0.5

qtable=dict() #key:(state,action) tuple, value: q-value

rndply= RandomPlayer('X')
qply=QLearningAgent('O')
results=[0,0,0] #draws,player1 wins,player2 wins
games=5_000

for i in tqdm(range(games)):
    board=[' ',' ',' ',' ',' ',' ',' ',' ',' ']
    player=1 if games<games//2 else 0
    while check_win(board)==False and board.count(' ')>0:
        if player==1:
            # completly random Q-learning agent
            board=qply.make_move(board)
        else:
            board=rndply.make_move(board)
        player=1-player
    if board.count(' ')==0:
        results[0]+=1
    else:
        results[player+1]+=1
print(results)
print("Q-learning agent win rate:",results[2]/games*100,"%")
print("Random player win rate:",results[1]/games*100,"%")
print("Draw rate:",results[0]/games*100,"%")
print("qtable size:",len(qtable))

  0%|          | 0/5000 [00:00<?, ?it/s]

[701, 2178, 2121]
Q-learning agent win rate: 42.42 %
Random player win rate: 43.56 %
Draw rate: 14.02 %
qtable size: 9040


# Play against human

In [8]:
class HumanPlayer():
    def __init__(self, symbol):
        self.symbol = symbol

    def make_move(self, state):
        while True:
            action = int(input("Choose your move, avaible moves are: " + str(np.where(np.array(state) == ' ')[0]) + "\n"))
            if state[action] == ' ':
                state[action] = self.symbol
                return state
            else:
                print("Invalid move!")

In [35]:
qply=QLearningAgent('O')
hlpy= HumanPlayer('X')
player = 1
board=[' ',' ',' ',' ',' ',' ',' ',' ',' ']
while check_win(board)==False and board.count(' ')>0:
    if player==1:
        # Q-learning agent
        board=qply.use_only_qtable(board)
    else:
        board=hlpy.make_move(board)
    player=1-player
    print_board(board)

if board.count(' ')==0:
    print("Draw!")
if player==1:
    print("Human player won!")
elif player==0:
    print("Q-learning won!")



[' ', ' ', ' ']
[' ', ' ', ' ']
['O', ' ', ' ']
-----------------
['X', ' ', ' ']
[' ', ' ', ' ']
['O', ' ', ' ']
-----------------
['X', ' ', ' ']
[' ', ' ', 'O']
['O', ' ', ' ']
-----------------
Invalid move!
['X', ' ', ' ']
[' ', 'X', 'O']
['O', ' ', ' ']
-----------------
['X', ' ', 'O']
[' ', 'X', 'O']
['O', ' ', ' ']
-----------------
Invalid move!
['X', ' ', 'O']
[' ', 'X', 'O']
['O', ' ', 'X']
-----------------
Human player won!
